In [0]:
from google.colab import drive
drive.mount('/content/drive/', timeout_ms=10000000)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import os

os.chdir("drive/My Drive/ColabNotebooks/NST/images")
print(os.listdir())

['content.jpg', 'style.jpg', 'generated']


In [0]:
import numpy as np
import keras.backend as K
from scipy.misc import imsave, imresize
from scipy.optimize import fmin_l_bfgs_b, fmin_tnc, fmin_cobyla, fmin_slsqp
from keras.applications import vgg19
from keras.preprocessing.image import load_img, img_to_array
import time
import urllib.request

Using TensorFlow backend.


In [0]:
url = 'https://images-na.ssl-images-amazon.com/images/I/61t8isaMjZL._SX331_BO1,204,203,200_.jpg'  
urllib.request.urlretrieve(url, 'style.jpg')

('style.jpg', <http.client.HTTPMessage at 0x7f1e2b2716a0>)

In [0]:
def preprocess(img_path):
    img = load_img(img_path)
    img = img_to_array(img)
    img = np.rot90(img)
    img = imresize(img, (img_h, img_w, 3)).astype('float64')
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)

    return img

In [0]:
def deprocess_image(img):
    img = img.reshape((img_h, img_w, 3))

    # subtract mean RGB computed on ImgeNet DB
    img[:, :, 0] += 103.939
    img[:, :, 1] += 116.779
    img[:, :, 2] += 123.68
    img = img[:, :, ::-1]
    img = np.clip(img, 0, 255).astype('float64')
    return img

In [0]:
def content_loss(content, gen):
    return K.sum(K.square(gen - content))

def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, gen): # find the Euclidean distance between the gram matrices of the feature maps
    S = gram_matrix(style)
    G = gram_matrix(gen)
    channels = 3
    size = img_h * img_w
    return K.sum(K.square(S - G)/(4 * (channels**2) * (size**2)))

In [0]:
def total_variation_loss(x): # to smooth the generated image
    a = K.square(x[:, :img_h - 1, :img_w - 1, :] - x[:, 1:, :img_w - 1, :])
    b = K.square(x[:, :img_h - 1, :img_w - 1, :] - x[:, :img_h - 1, 1:, :])

    return K.sum(K.pow(a+b, 1.25))

In [0]:
CONTENT_IMG_PATH = "content.jpg"
STYLE_IMG_PATH = "style.jpg"

In [0]:
h, w = load_img(CONTENT_IMG_PATH).size
img_h = 400
img_w = int(h*img_h / w)

In [0]:
content_img = K.variable(preprocess(CONTENT_IMG_PATH))
style_img = K.variable(preprocess(STYLE_IMG_PATH))
gen_img = K.placeholder(shape=(1, img_h, img_w, 3))

input_tensor = K.concatenate([content_img, style_img, gen_img], axis=0)

model = vgg19.VGG19(include_top=False, weights='imagenet', input_tensor=input_tensor)
print('Model Loaded...')
print(model.summary())

output_dict = dict([(layer.name, layer.output) for layer in model.layers])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """


Model Loaded...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0    

In [0]:
CONTENT_WEIGHT = 1e2
STYLE_WEIGHT = 1e4
TV_WEIGHT = 1e3

In [0]:
loss = 0

# compute content loss
layer_features = output_dict['block2_conv2'] # https://arxiv.org/abs/1603.08155
content_img_features = layer_features[0, :, :, :]
gen_img_features = layer_features[2, :, :, :]
loss += CONTENT_WEIGHT * content_loss(content_img_features, gen_img_features)

# compute style loss
feature_layer_names = ['block1_conv2', 'block2_conv2', 'block3_conv4', 'block4_conv4', 'block5_conv4'] # https://arxiv.org/abs/1603.08155

for name in feature_layer_names:
    layer_features = output_dict[name]
    style_features = layer_features[1, :, :, :]
    gen_img_features = layer_features[2, :, :, :]
    loss += (STYLE_WEIGHT / len(feature_layer_names)) * style_loss(style_features, gen_img_features)

# compute total variation loss
loss += TV_WEIGHT * total_variation_loss(gen_img) # reduce the amount of noise in the generated image

In [0]:
grads = K.gradients(loss, gen_img)
f_output = K.function([gen_img], [loss] + grads) # calculate the loss and the gradients during the optimization

In [0]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_h, img_w, 3))

    global f_output
    outs = f_output([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')

    return loss_value, grad_values

class Evaluator:

    def __init__(self):
        self.loss_value = None
        self.grad_value = None

    def loss(self, x):
        assert self.loss_value is None
        self.loss_value, self.grad_value = eval_loss_and_grads(x)
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grads_values = np.copy(self.grad_value)
        self.loss_value = None
        self.grad_value = None
        return grads_values

In [0]:
ITER = 100

evaluator = Evaluator()
x = np.random.uniform(0, 255, (1, img_h, img_w, 3)) - 128.

for i in range(ITER):
    start_time = time.time()
    print('step {} ==> '.format(i), end='')
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(), fprime=evaluator.grads, maxfun=100)
    print('loss: {},'.format(min_val), end='')
    img = deprocess_image(x)
    imsave('generated/generated_img{}.jpg'.format(i), img)
    print(' Image saved. time: {}'.format(time.time() - start_time))

step 0 ==> loss: 150340733239296.0, Image saved. time: 55.3421049118042
step 1 ==> 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  if sys.path[0] == '':


loss: 125997311590400.0, Image saved. time: 53.718560457229614
step 2 ==> loss: 123731582124032.0, Image saved. time: 53.7633740901947
step 3 ==> loss: 121677665009664.0, Image saved. time: 53.58441495895386
step 4 ==> loss: 119260504391680.0, Image saved. time: 53.86927795410156
step 5 ==> loss: 121785643171840.0, Image saved. time: 53.780231952667236
step 6 ==> loss: 123780504485888.0, Image saved. time: 53.80539608001709
step 7 ==> loss: 121561952550912.0, Image saved. time: 54.141886949539185
step 8 ==> loss: 117689200672768.0, Image saved. time: 53.825281858444214
step 9 ==> loss: 120980236140544.0, Image saved. time: 53.798566579818726
step 10 ==> loss: 117266154782720.0, Image saved. time: 53.66770648956299
step 11 ==> loss: 115107480731648.0, Image saved. time: 53.87474536895752
step 12 ==> loss: 121357354401792.0, Image saved. time: 53.84250092506409
step 13 ==> loss: 115798743973888.0, Image saved. time: 54.03455471992493
step 14 ==> loss: 121086679187456.0, Image saved. time